In [ ]:
!apt-get update
!apt-get install libgdal-dev gdal-bin python-gdal python-numpy python-scipy -y
!pip install wget
!pip install azure-storage-blob
!pip install pyproj

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta

import tempfile
import wget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gdal
import osr

from azure.storage.blob import ContainerClient
from pyproj import Proj

## Import Base Data Files

In [ ]:
ground_measures_metadata = pd.read_csv('/content/drive/MyDrive/snocast/eval/data/ground_measures_metadata.csv')
submission_format = pd.read_csv('/content/drive/MyDrive/snocast/eval/data/submission_format.csv')
run_date = '2022-06-30'
lookback = 15

In [ ]:
# get latitude longitude for grids
f = open('/content/drive/MyDrive/snocast/eval/data/grid_cells.geojson')
grid_cells = json.load(f)
print('length grid_cells features: ', len(grid_cells['features']))

ids = []
lats = []
lons = []
bboxes = []
coords = []

for grid_cell in grid_cells['features']:
    cell_id = grid_cell['properties']['cell_id']
    coordinates = grid_cell['geometry']['coordinates'][0][1:]
    lon, lat = np.mean(coordinates, axis=0)
    northeast_corner = np.max(coordinates, axis=0)
    southwest_corner = np.min(coordinates, axis=0)
    # bbox = [min_lon, min_lat, max_lon, max_lat]
    bbox = np.concatenate([southwest_corner,northeast_corner])
    ids.append(cell_id)
    lats.append(lat)
    lons.append(lon)
    bboxes.append(bbox)
    coords.append(coordinates)

grid_cells_pd = pd.DataFrame({'location_id': ids, 
                             'latitude': lats, 
                             'longitude': lons, 
                             'bbox': bboxes,
                             'coordinates': coords})

### Setup Environment to Download Modis Data

In [ ]:
modis_account_name = 'modissa'
modis_container_name = 'modis-006'
modis_account_url = 'https://' + modis_account_name + '.blob.core.windows.net/'
modis_blob_root = modis_account_url + modis_container_name + '/'

# This file is provided by NASA; it indicates the lat/lon extents of each
# MODIS tile.
#
# The file originally comes from:
#
# https://modis-land.gsfc.nasa.gov/pdf/sn_bound_10deg.txt
modis_tile_extents_url = modis_blob_root + 'sn_bound_10deg.txt'

temp_dir = os.path.join(tempfile.gettempdir(),'modis')
os.makedirs(temp_dir,exist_ok=True)
fn = os.path.join(temp_dir,modis_tile_extents_url.split('/')[-1])
wget.download(modis_tile_extents_url, fn)

# Load this file into a table, where each row is (v,h,lonmin,lonmax,latmin,latmax)
modis_tile_extents = np.genfromtxt(fn,
                     skip_header = 7, 
                     skip_footer = 3)

# modis_container_name = 'modis-061'
# modis_account_url = 'https://' + modis_account_name + '.blob.core.windows.net/'
# modis_blob_root = modis_account_url + modis_container_name + '/'

modis_container_client = ContainerClient(account_url=modis_account_url, 
                                         container_name=modis_container_name,
                                         credential=None)

### Functions

In [ ]:
# https://gis.stackexchange.com/questions/265400/getting-tile-number-of-sinusoidal-modis-product-from-lat-long
CELLS = 2400
VERTICAL_TILES = 18
HORIZONTAL_TILES = 36
EARTH_RADIUS = 6371007.181
EARTH_WIDTH = 2 * np.pi * EARTH_RADIUS

MODIS_GRID = Proj(f'+proj=sinu +R={EARTH_RADIUS} +nadgrids=@null +wktext')

TILE_WIDTH = EARTH_WIDTH / HORIZONTAL_TILES
TILE_HEIGHT = TILE_WIDTH
CELL_SIZE = TILE_WIDTH / CELLS
def lat_lon_to_modis_tile(lat, lon):
    x, y = MODIS_GRID(lon, lat)
    h = (EARTH_WIDTH * .5 + x) / TILE_WIDTH
    v = -(EARTH_WIDTH * .25 + y - (VERTICAL_TILES - 0) * TILE_HEIGHT) / TILE_HEIGHT
    return int(h), int(v)


def list_blobs_in_folder(container_name,folder_name):
    """
    List all blobs in a virtual folder in an Azure blob container
    """
    
    files = []
    generator = modis_container_client.list_blobs(name_starts_with=folder_name)
    for blob in generator:
        files.append(blob.name)
    return files
        
    
def list_hdf_blobs_in_folder(container_name,folder_name):
    """"
    List .hdf files in a folder
    """
    
    files = list_blobs_in_folder(container_name,folder_name)
    files = [fn for fn in files if fn.endswith('.hdf')]
    return files 

def clip_nan(value):
  if value > 100:
    return np.nan
  return value

def get_modis_tile_dataset(product, h, v, daynum):
  folder = product + '/' + '{:0>2d}/{:0>2d}'.format(h,v) + '/' + daynum

  # Find all HDF files from this tile on this day
  filenames = list_hdf_blobs_in_folder(modis_container_name, folder)
  print('Found {} matching file(s):'.format(len(filenames)))
  for fn in filenames:
      print(fn)

  # Work with the first returned URL
  blob_name = filenames[0]

  # Download to a temporary file
  url = modis_blob_root + blob_name

  fn = os.path.join(data_dir,blob_name.replace('/','_'))
  if not os.path.isfile(fn):
      wget.download(url,fn)

  ds = gdal.Open(fn, gdal.GA_ReadOnly)

  return ds

def get_ndsi_value(dataset, x, y):
  # https://gis.stackexchange.com/questions/221292/retrieve-pixel-value-with-geographic-coordinate-as-input-with-gdal
  cols = dataset.RasterXSize
  rows = dataset.RasterYSize

  transform = dataset.GetGeoTransform()

  xOrigin = transform[0]
  yOrigin = transform[3]
  pixelWidth = transform[1]
  pixelHeight = -transform[5]

  data = dataset.ReadAsArray(0, 0, cols, rows)

  data_col = int((x - xOrigin) / pixelWidth)
  data_row = int((yOrigin - y) / pixelHeight)

  try:
    ndsi_value = data[data_row][data_col]
  except:
    print('exception')
    ndsi_value = np.nan

  return clip_nan(ndsi_value)

def modis_projection_func(dataset):
        srs_wkt = dataset.GetProjection()  # gives SRS in WKT
        srs_converter = osr.SpatialReference()  # makes an empty spatial ref object
        srs_converter.ImportFromWkt(srs_wkt)  # populates the spatial ref object with our WKT SRS
        projection = srs_converter.ExportToProj4()
        p_modis_grid = Proj(projection) # '+proj=sinu +R=6371007.181 +nadgrids=@null +wktext'
        return p_modis_grid

### Access and Plot Modis Tile

In [ ]:
cell_modis_tiles_h = []
cell_modis_tiles_v = []
for idx, row in grid_cells_pd.iterrows():
  lat, lon = row[['latitude','longitude']].values
  h, v = lat_lon_to_modis_tile(lat, lon)
  cell_modis_tiles_h.append(h)
  cell_modis_tiles_v.append(v)


In [ ]:
grid_cells_pd['h_tile'] = cell_modis_tiles_h
grid_cells_pd['v_tile'] = cell_modis_tiles_v

In [ ]:
grid_cells_pd.groupby(['h_tile','v_tile']).count()

## Get NDSI_Snow_Cover

In [ ]:
max_date = datetime.strptime(run_date,'%Y-%m-%d')
date_list = [(max_date - timedelta(days=x)).strftime('%Y-%m-%d') for x in range(lookback+1)]
print(date_list)
tile_list = [(8,4), (8,5), (9,4), (9,5), (10,4)]

If this is the first time this batch job is run for a given `run_date` then it will pull data for all of the dates in the date list.

If this batch job has been previously run for a given `run_date`, then the job will look for existing files and pull data from where those files left off.

In [ ]:
fresh_run = True
try:
  terra_df = pd.read_parquet(f'/content/drive/MyDrive/snocast/eval/data/modis/modis_terra_pc_{run_date}.parquet')
  aqua_df = pd.read_parquet(f'/content/drive/MyDrive/snocast/eval/data/modis/modis_aqua_pc_{run_date}.parquet')
  max_terra = terra_df.date.max()
  max_aqua = aqua_df.date.max()
  print("Max Terra: ", max_terra, "Min Aqua: ", max_aqua)
  terra_date_list = [d for d in date_list if d > max_terra]
  aqua_date_list = [d for d in date_list if d > max_aqua]
  fresh_run = False
except:
  print("Fresh Run")
  terra_date_list = date_list
  aqua_date_list = date_list

In [ ]:
data_dir = '/content/drive/MyDrive/snocast/eval/data/modis_data_raw'

terra_list = []
terra_ids = []
terra_dates = []
terra_lats = []
terra_lons = []

aqua_list = []
aqua_ids = []
aqua_dates = []
aqua_lats = []
aqua_lons = []

for pull_date in date_list:
  print(pull_date)
  for tile in tile_list:
    h, v = tile
    print(h, v)
    daynum = run_date[0:4] + datetime.strptime(pull_date, '%Y-%m-%d').strftime('%j')
    tile_grid_cells = grid_cells_pd[(grid_cells_pd['h_tile'] == h) & (grid_cells_pd['v_tile'] == v)]

    get_terra = False
    get_aqua = False

    if pull_date in terra_date_list:
      try:
        terra_ds = get_modis_tile_dataset('MOD10A1', h, v, daynum)
        terra_snow_cover = gdal.Open(terra_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
        p_modis_grid = modis_projection_func(terra_snow_cover)
        get_terra = True
      except:
        print(f'No Terra file for {h}, {v}, {pull_date}')


    if pull_date in aqua_date_list:
      try:
        aqua_ds = get_modis_tile_dataset('MYD10A1', h, v, daynum)
        aqua_snow_cover = gdal.Open(aqua_ds.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
        p_modis_grid = modis_projection_func(aqua_snow_cover)
        get_aqua = True
      except:
        print(f'No Aqua file for {h}, {v}, {pull_date}')


    if get_terra or get_aqua:
      for idx, row in tile_grid_cells.iterrows():
        for lon, lat in row['coordinates']+[[row['longitude'], row['latitude']]]:
          x, y = p_modis_grid(lon, lat)

          if get_terra:
            terra_list.append(get_ndsi_value(terra_snow_cover, x, y))
            terra_ids.append(row['location_id'])
            terra_dates.append(pull_date)
            terra_lats.append(lat)
            terra_lons.append(lon)

          if get_aqua:
            aqua_list.append(get_ndsi_value(aqua_snow_cover, x, y))
            aqua_ids.append(row['location_id'])
            aqua_dates.append(pull_date)
            aqua_lats.append(lat)
            aqua_lons.append(lon)

In [ ]:
if fresh_run:
  terra_df = pd.DataFrame({'NDSI_Snow_Cover': terra_list,
                          'location_id': terra_ids,
                          'date': terra_dates,
                          'latitude': terra_lats,
                          'longitude': terra_lons})

  aqua_df = pd.DataFrame({'NDSI_Snow_Cover': aqua_list,
                          'location_id': aqua_ids,
                          'date': aqua_dates,
                          'latitude': aqua_lats,
                          'longitude': aqua_lons})
else:
  if len(terra_list) > 0:
    print("new Terra data!")
    new_terra_df = pd.DataFrame({'NDSI_Snow_Cover': terra_list,
                            'location_id': terra_ids,
                            'date': terra_dates,
                            'latitude': terra_lats,
                            'longitude': terra_lons})
    
    terra_df = pd.concat([terra_df, new_terra_df])

  if len(aqua_list) > 0:
    print("new Aqua data!")
    new_aqua_df = pd.DataFrame({'NDSI_Snow_Cover': aqua_list,
                            'location_id': aqua_ids,
                            'date': aqua_dates,
                            'latitude': aqua_lats,
                            'longitude': aqua_lons})
    aqua_df = pd.concat([aqua_df, new_aqua_df])

In [ ]:
terra_df.groupby('date').count()

In [ ]:
aqua_df.groupby('date').count()

In [ ]:
terra_df.to_parquet(f'/content/drive/MyDrive/snocast/eval/data/modis/modis_terra_pc_{run_date}.parquet')
aqua_df.to_parquet(f'/content/drive/MyDrive/snocast/eval/data/modis/modis_aqua_pc_{run_date}.parquet')